# Node Classification for Heterogeneous Graph
This notebook introduces how to use AutoGL to automate the learning of heterogeneous graphs in Deep Graph Library (DGL).


## Import libraries

In [1]:
# Import needed libraries.
import torch
from autogl.datasets import build_dataset_from_name
from autogl.module.model.dgl import AutoHAN
from sklearn.metrics import f1_score
from autogl.solver import AutoHeteroNodeClassifier
from tqdm import tqdm

## Creating a Heterogeneous Graph

AutoGL supports datasets created in DGL. We provide two datasets named "hetero-acm-han" and "hetero-acm-hgt" for HAN and HGT models, respectively [1].

The following code snippet is an example for loading a heterogeneous graph. You can also access to data stored in the dataset object for more details:

In [2]:
# load dataset
dataset = build_dataset_from_name("hetero-acm-han")
g = dataset[0]

# get dataset information
node_type = dataset.schema["target_node_type"]
labels = g.nodes[node_type].data['label']
num_classes = labels.max().item() + 1
num_features=g.nodes[node_type].data['feat'].shape[1]

# get train-val-test mask
train_mask = g.nodes[node_type].data['train_mask']
val_mask = g.nodes[node_type].data['val_mask']
test_mask = g.nodes[node_type].data['test_mask']

Using cached file /villa/xbn/.cache-autogl/data/hetero-acm-han/raw/ACM.mat


In [14]:
# set device
device = 'cpu' # or cuda:0

You can also build your own dataset and do feature engineering by adding files in the location AutoGL/autogl/datasets/_heterogeneous_datasets/_dgl_heterogeneous_datasets.py. 

We suggest users create a data object of type `torch_geometric.data.HeteroData` refering to the official documentation of DGL.


## Building Heterogeneous GNN Modules

AutoGL integrates commonly used heterogeneous graph neural network models such as ``HeteroRGCN`` (Schlichtkrull et al., 2018) [2], ``HAN`` (Wang et al., 2019) [3] and ``HGT`` (Hu et al., 2029) [4].


In [3]:
# initialize auto model
model = AutoHAN(
    dataset=dataset,
    num_features=num_features,
    num_classes=num_classes,
    device=device,
    init=True
).model

Then you can train the model for 100 epochs.

In [4]:
# Define the loss function.
loss_fcn = torch.nn.CrossEntropyLoss()
# Define the loss optimizer.
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2,
                                weight_decay=1e-2)


# Define the evlaute score function.
def score(logits, labels):
    _, indices = torch.max(logits, dim=1)
    prediction = indices.long().cpu().numpy()
    labels = labels.cpu().numpy()

    accuracy = (prediction == labels).sum() / len(prediction)
    micro_f1 = f1_score(labels, prediction, average='micro')
    macro_f1 = f1_score(labels, prediction, average='macro')

    return accuracy, micro_f1, macro_f1

# Define the Evaluation function.
def evaluate(model, g, labels, mask, loss_func):
    model.eval()
    with torch.no_grad():
        logits = model(g)
    loss = loss_func(logits[mask], labels[mask])
    accuracy, micro_f1, macro_f1 = score(logits[mask], labels[mask])

    return loss, accuracy, micro_f1, macro_f1

Finally, evaluate the model.

In [6]:

# Training.
g = g.to(device)
for epoch in tqdm(range(100)):
    model.train()
    logits = model(g)
    loss = loss_fcn(logits[train_mask], labels[train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    val_loss, val_acc, _, _ = evaluate(model, g, labels, val_mask, loss_fcn)

    
_, test_acc, _, _ = evaluate(model, g, labels, test_mask, loss_fcn)
print('test accuracy for HAN: {:.4f}'.format(test_acc))

100%|██████████| 100/100 [20:36<00:00, 12.37s/it]


test accuracy for HAN: 0.8988


You can also define your own heterogeneous graph neural network models by adding files in the location AutoGL/autogl/module/model/dgl/hetero.

## Automatic Search for Node Classification Tasks

On top of the modules mentioned above, we provide a high-level API Solver to control the overall pipeline. We encapsulated the training process in the Building Heterogeneous GNN Modules part in the solver AutoHeteroNodeClassifier that supports automatic hyperparametric optimization as well as feature engineering and ensemble.
In this part, we will show you how to use AutoHeteroNodeClassifier to automatically predict the publishing conference of a paper using the ACM academic graph dataset.

Firstly, we get the pre-defined model hyperparameter. 

In [15]:
model_hp = {
            "num_layers": 2,
            "hidden": [256], 
            "heads": [8], 
            "dropout": 0.2,
            "act": "gelu",
        }

Secondly, use AutoHeteroNodeClassifier directly to bulid automatic heterogeneous GNN models in the following example:

In [16]:
def fixed(**kwargs):
    return [{
        'parameterName': k,
        "type": "FIXED",
        "value": v
    } for k, v in kwargs.items()]
    
solver = AutoHeteroNodeClassifier(
            graph_models=["han"],
            hpo_module="random",
            ensemble_module=None,
            max_evals=1,
            device=device,
            trainer_hp_space=fixed(
                max_epoch=100,
                early_stopping_round=101,
                lr=1e-3,
                weight_decay=1e-2
            ),
            model_hp_spaces=[fixed(**model_hp)]
        )

Finally, fit and evlauate the model.

In [17]:
solver.fit(dataset)
acc = solver.evaluate()

print('test accuracy for HAN: {:.4f}'.format(acc))

HPO Search Phase:



  0%|          | 0/1 [00:00<?, ?it/s]WARNING:root:Ignore passed dec since enc is a whole model


[2022-10-25 13:39:08] WARNING (root/MainThread) Ignore passed dec since enc is a whole model


INFO:HPO:Parameter: {'trainer': {'max_epoch': 100, 'early_stopping_round': 101, 'lr': 0.001, 'weight_decay': 0.01}, 'encoder': {'num_layers': 2, 'hidden': [256], 'heads': [8], 'dropout': 0.2, 'act': 'gelu'}, 'decoder': {}} acc: 0.4937655860349127 higher_better


[2022-10-25 14:31:24] INFO (HPO/MainThread) Parameter: {'trainer': {'max_epoch': 100, 'early_stopping_round': 101, 'lr': 0.001, 'weight_decay': 0.01}, 'encoder': {'num_layers': 2, 'hidden': [256], 'heads': [8], 'dropout': 0.2, 'act': 'gelu'}, 'decoder': {}} acc: 0.4937655860349127 higher_better


...

INFO:HPO:Parameter: {'trainer': {'max_epoch': 100, 'early_stopping_round': 101, 'lr': 0.001, 'weight_decay': 0.01}, 'encoder': {'num_layers': 2, 'hidden': [256], 'heads': [8], 'dropout': 0.2, 'act': 'gelu'}, 'decoder': {}} acc: 0.4937655860349127 higher_better


[2022-10-25 14:31:24] INFO (HPO/MainThread) Parameter: {'trainer': {'max_epoch': 100, 'early_stopping_round': 101, 'lr': 0.001, 'weight_decay': 0.01}, 'encoder': {'num_layers': 2, 'hidden': [256], 'heads': [8], 'dropout': 0.2, 'act': 'gelu'}, 'decoder': {}} acc: 0.4937655860349127 higher_better


INFO:HeteroNodeClassifier:Ensemble argument on, will try using ensemble model.


[2022-10-25 14:31:24] INFO (HeteroNodeClassifier/MainThread) Ensemble argument on, will try using ensemble model.


[2022-10-25 14:31:24] WARNING (HeteroNodeClassifier/MainThread) Cannot use ensemble because no ensebmle module is given. Will use best model instead.
test accuracy for HAN: 0.4957



References:

[1] https://data.dgl.ai/dataset/ACM.mat

[2] Schlichtkrull, Michael, et al. "Modeling relational data with graph convolutional networks." European semantic web conference. Springer, Cham, 2018.

[3] Wang, Xiao, et al. "Heterogeneous graph attention network." The World Wide Web Conference. 2019.

[4] Yun, Seongjun, et al. "Graph transformer networks." Advances in Neural Information Processing Systems 32 (2019): 11983-11993.